In [1]:
%cd ~/user_data/research/git/zero-shot-object-detection/frontend_owlvit/owlv2_coco_base
%pwd

/mnt/researchfiles/ECE IMAPLE/cluster_data/user_data/jw3897/research/git/zero-shot-object-detection/frontend_owlvit/owlv2_coco_base


'/mnt/researchfiles/ECE IMAPLE/cluster_data/user_data/jw3897/research/git/zero-shot-object-detection/frontend_owlvit/owlv2_coco_base'

In [2]:
import os
import random
import numpy as np
import torch
from torch.utils.data import DataLoader
import torchvision
from torchvision.datasets import CocoDetection
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
from PIL import Image
import cv2
from pycocotools.coco import COCO
from transformers import pipeline
from transformers import Owlv2ImageProcessor, Owlv2Processor, Owlv2ForObjectDetection
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from tqdm import tqdm
import pandas as pd
import glob

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

/home/DREXEL/jw3897/miniconda3/envs/owlvit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
val_image_dir = r'/mnt/active_storage/Joe/coco_set/val2017/images'
val_annotation_path = r'/mnt/active_storage/Joe/coco_set/val2017/annotations/ovd_ins_val2017_b.json'

coco_val_dataset = CocoDetection(
    root=val_image_dir, 
    annFile=val_annotation_path,
)

loading annotations into memory...
Done (t=0.45s)
creating index...
index created!


In [4]:
ovd_base_image_ids = []
for item in coco_val_dataset:
    ovd_base_image_ids.append(item[1][0]['image_id'])

print(len(ovd_base_image_ids))

4533


In [5]:
predictions_df_total = pd.read_parquet(r'./owlv2_coco_base_predictions.parquet')

In [6]:
objectness_threshold = 0.2
score_threshold = 0.1

predictions_likely_objects = predictions_df_total[predictions_df_total['objectness_score'] >= objectness_threshold].sort_values('objectness_score', ascending=False).groupby('image_id').head(100).sort_values('image_id')
predictions_likely_objects = predictions_likely_objects.loc[predictions_likely_objects['image_id'].isin(ovd_base_image_ids)]

known_objects = predictions_likely_objects[predictions_likely_objects['score'] >= score_threshold]
unknown_objects = predictions_likely_objects[predictions_likely_objects['score'] <= score_threshold]

display(known_objects)
display(unknown_objects)

print(f"Number of Known Annotations: {len(known_objects)}. Number of Unknown Annotations: {len(unknown_objects)}")

,image_id,label,score,objectness_score,xmin,ymin,xmax,ymax
1233,139,vase,0.351422,0.227661,350.876923,209.574448,362.114807,230.386261
1336,139,vase,0.334456,0.284153,166.681076,232.826904,186.017776,266.831390
1150,139,tv,0.704314,0.542719,5.215874,166.488068,154.236847,267.109009
1163,139,vase,0.248859,0.221027,240.760803,197.673264,253.225784,213.093719
1253,139,tv,0.409905,0.306233,559.908569,209.109192,640.466003,289.297943
...,...,...,...,...,...,...,...,...
178952,581781,orange,0.176230,0.327590,281.273376,434.908295,371.029541,478.781433
178161,581781,banana,0.284449,0.241578,201.138290,274.905090,270.746216,331.941162
177593,581781,banana,0.301017,0.333404,445.379303,101.757217,638.398376,269.523865
178102,581781,banana,0.323884,0.339627,141.406525,184.186020,441.437103,460.155182


,image_id,label,score,objectness_score,xmin,ymin,xmax,ymax
1732,139,vase,0.048069,0.234920,555.129150,290.876068,577.775818,331.965332
2270,139,chair,0.019408,0.247597,466.036926,354.445007,637.800171,425.332062
1053,139,vase,0.016014,0.221426,337.896454,176.377960,381.800262,222.027878
1465,139,zebra,0.002407,0.654636,217.288071,232.345657,296.701080,303.250427
203,139,clock,0.010626,0.217656,218.589600,0.239856,320.863190,62.722061
...,...,...,...,...,...,...,...,...
176913,581781,banana,0.013458,0.320759,331.135406,72.229202,381.259827,136.007263
176729,581781,orange,0.012246,0.347357,296.637360,30.675127,337.894287,85.237419
177278,581781,banana,0.030917,0.268310,388.986481,123.088112,447.452820,163.840027
177148,581781,orange,0.019083,0.268582,289.337677,123.226250,324.085205,156.411377


Number of Known Annotations: 33571. Number of Unknown Annotations: 27387


In [7]:
# image_id | [known object labels] | [known object locations] | unknown object location

In [8]:
# image_id | [known object labels] | [known object locations] | unknown object location

In [9]:
final_df = pd.DataFrame(columns=[
    'image_id', 
    'known_object_labels', 
    'known_object_bboxes',
    'unknown_object_bbox',
    'unknown_object_potential_label',
    'unknown_object_class_score',
    'unknown_object_objectness'
    ])

#merged_df = pd.merge(predictions_likely_objects, known_objects)

In [14]:
dfs = []

#unknown_object = unknown_objects.iloc[0]
for idx, unknown_object in unknown_objects.iterrows():
    known_objects_in_image = known_objects[known_objects['image_id'] == unknown_object['image_id']]
    unknown_object_dict = {
        'image_id': unknown_object['image_id'],
    }

    known_object_labels = []
    known_object_bboxes = []
    for idx, known_object in known_objects_in_image.iterrows():
        known_object_labels.append(known_object['label'])
        known_object_bboxes.append([known_object['xmin'], known_object['ymin'], known_object['xmax'], known_object['ymax']])

    unknown_object_bbox = [unknown_object['xmin'], unknown_object['ymin'], unknown_object['xmax'], unknown_object['ymax']]

    unknown_object_dict['known_object_labels'] = known_object_labels
    unknown_object_dict['known_object_bboxes'] = known_object_bboxes
    unknown_object_dict['unknown_object_bbox'] = unknown_object_bbox
    unknown_object_dict['unknown_object_potential_label'] = unknown_object['label']
    unknown_object_dict['unknown_class_score'] = unknown_object['score']
    unknown_object_dict['unknown_object_objectness'] = unknown_object['objectness_score']


    df = pd.DataFrame(columns=[
    'image_id', 
    'known_object_labels', 
    'known_object_bboxes',
    'unknown_object_bbox',
    'unknown_object_potential_label',
    'unknown_object_class_score',
    'unknown_object_objectness'
    ])

    df.loc[0] = [
        unknown_object_dict['image_id'], 
        unknown_object_dict['known_object_labels'], 
        unknown_object_dict['known_object_bboxes'], 
        unknown_object_dict['unknown_object_bbox'], 
        unknown_object_dict['unknown_object_potential_label'],
        unknown_object_dict['unknown_class_score'],
        unknown_object_dict['unknown_object_objectness']
        ]

    dfs.append(df)


final_df = pd.concat(dfs)

In [15]:
print(f"Objectness Threshold: {objectness_threshold}; Score Threshold: {score_threshold}")
display(final_df)

Objectness Threshold: 0.2; Score Threshold: 0.1


,image_id,known_object_labels,known_object_bboxes,unknown_object_bbox,unknown_object_potential_label,unknown_object_class_score,unknown_object_objectness
0,139,"[vase, vase, tv, vase, tv, vase]","[[350.8769226074219, 209.57444763183594, 362.1...","[555.129150390625, 290.8760681152344, 577.7758...",vase,0.048069,0.234920
0,139,"[vase, vase, tv, vase, tv, vase]","[[350.8769226074219, 209.57444763183594, 362.1...","[466.03692626953125, 354.44500732421875, 637.8...",chair,0.019408,0.247597
0,139,"[vase, vase, tv, vase, tv, vase]","[[350.8769226074219, 209.57444763183594, 362.1...","[337.8964538574219, 176.37796020507812, 381.80...",vase,0.016014,0.221426
0,139,"[vase, vase, tv, vase, tv, vase]","[[350.8769226074219, 209.57444763183594, 362.1...","[217.28807067871094, 232.3456573486328, 296.70...",zebra,0.002407,0.654636
0,139,"[vase, vase, tv, vase, tv, vase]","[[350.8769226074219, 209.57444763183594, 362.1...","[218.589599609375, 0.23985624313354492, 320.86...",clock,0.010626,0.217656
...,...,...,...,...,...,...,...
0,581781,"[banana, banana, banana, banana, banana, banan...","[[463.3008117675781, 422.8099365234375, 592.42...","[331.1354064941406, 72.22920227050781, 381.259...",banana,0.013458,0.320759
0,581781,"[banana, banana, banana, banana, banana, banan...","[[463.3008117675781, 422.8099365234375, 592.42...","[296.6373596191406, 30.675127029418945, 337.89...",orange,0.012246,0.347357
0,581781,"[banana, banana, banana, banana, banana, banan...","[[463.3008117675781, 422.8099365234375, 592.42...","[388.9864807128906, 123.0881118774414, 447.452...",banana,0.030917,0.268310
0,581781,"[banana, banana, banana, banana, banana, banan...","[[463.3008117675781, 422.8099365234375, 592.42...","[289.3376770019531, 123.22624969482422, 324.08...",orange,0.019083,0.268582


In [16]:
final_df.to_parquet(r'/mnt/archive/owlvit_results/coco_base_prepared_for_adam_obj2_score1.parquet')